In [1]:
##########################
# 配置运行环境
##########################

%matplotlib inline

import numpy as np
import pandas as pd
from matplotlib import pyplot

# matplotlib 对中文的支持及保存为 pdf 格式
from matplotlib import font_manager
cn_font = font_manager.FontProperties(fname='msyh.ttf', size=16)  # 网上支持中文

from matplotlib import rcParams
# rcParams['font.family'] = 'Microsoft YaHei'  # 本地支持中文
rcParams['pdf.fonttype'] = 42
rcParams['figure.figsize'] = (8, 5)

# Look pretty...
from matplotlib import style
style.use('ggplot')

# 设置 numpy 的输出精度, 并且阻止使用科学记数法
np.set_printoptions(precision=6, suppress=True)

# 二元离散随机变量的条件分布

考虑下列随机变量 $W, I, X, Y$，其中 $W$ 和 $X$ 的取值为 {晴天, 雨天, 阴天}，$I$ 和 $Y$ 的取值为 $0,1$。它们的联合分布列如下

In [2]:
joint_prob_WI = pd.DataFrame([[0, 1/2], [1/6, 0], [1/3, 0]], columns=[0, 1], index=['晴天', '雨天', '阴天'])

joint_prob_XY = pd.DataFrame([[1/4, 1/4], [1/6, 1/6], [1/12, 1/12]], columns=[0, 1], index=['晴天', '雨天', '阴天'])

In [3]:
print('随机变量 W, I 的联合分布列为 \n', joint_prob_WI.head())

随机变量 W, I 的联合分布列为 
            0    1
晴天  0.000000  0.5
雨天  0.166667  0.0
阴天  0.333333  0.0


In [4]:
print('随机变量 X, Y 的联合分布列为 \n', joint_prob_XY.head())

随机变量 X, Y 的联合分布列为 
            0         1
晴天  0.250000  0.250000
雨天  0.166667  0.166667
阴天  0.083333  0.083333


计算下列条件概率
1. $p_{W|I}(\text{晴天}|1)$
2. $p_{X|Y}(\text{晴天}|1)$
3. $p_{I|W}(1|\text{阴天})$
4. $p_{Y|X}(1|\text{阴天})$

根据条件分布的定义，我们知道
$$
p_{W|I}(\text{晴天 }|\ 1) = \frac{p_{W,I}(\text{晴天},1)}{p_I(1)}
$$
可见，要计算随机变量的条件概率，先要计算边缘分布。

In [5]:
prob_W = joint_prob_WI.sum(axis=1)
prob_I = joint_prob_WI.sum(axis=0)
prob_X = joint_prob_XY.sum(axis=1)
prob_Y = joint_prob_XY.sum(axis=0)

print('随机变量 I 的边缘分布列为 {}'.format(prob_I.values))
print('随机变量 W 的边缘分布列为 {}'.format(prob_W.values))
print('随机变量 X 的边缘分布列为 {}'.format(prob_X.values))
print('随机变量 Y 的边缘分布列为 {}'.format(prob_Y.values))

随机变量 I 的边缘分布列为 [0.5 0.5]
随机变量 W 的边缘分布列为 [0.5      0.166667 0.333333]
随机变量 X 的边缘分布列为 [0.5      0.333333 0.166667]
随机变量 Y 的边缘分布列为 [0.5 0.5]


做好上述准备工作后，容易算得如下结果

In [6]:
print('第 1 问的概率为：', joint_prob_WI.loc['晴天', 1] / prob_I[1])
print('第 2 问的概率为：', joint_prob_XY.loc['晴天', 1] / prob_Y[1])
print('第 3 问的概率为：', joint_prob_WI.loc['阴天', 1] / prob_W['阴天'])
print('第 4 问的概率为：', joint_prob_XY.loc['阴天', 1] / prob_X['阴天'])

第 1 问的概率为： 1.0
第 2 问的概率为： 0.5000000000000001
第 3 问的概率为： 0.0
第 4 问的概率为： 0.5


## Simpson's Paradox

我们分析一个有关高校招生中是否存在性别歧视现象的奇怪结论。

假设 $G$ (性别 gender)，$D$ (院系 department)，$A$ (录取 admission) 是大学招生中涉及的三个随机变量。它们的联合分布列是一个三元数组。数据如下

In [7]:
data = {
        ('female', 'A', 'admitted'): 0.019566946531153304,
        ('female', 'A', 'rejected'): 0.004295183384887301,
        ('female', 'B', 'admitted'): 0.0037560760053027007,
        ('female', 'B', 'rejected'): 0.0017675651789660005,
        ('female', 'C', 'admitted'): 0.044547061422890007,
        ('female', 'C', 'rejected'): 0.086473707467962915,
        ('female', 'D', 'admitted'): 0.028999116217410508,
        ('female', 'D', 'rejected'): 0.053855501546619514,
        ('female', 'E', 'admitted'): 0.020839593460008802,
        ('female', 'E', 'rejected'): 0.065992045956694709,
        ('female', 'F', 'admitted'): 0.0052739726027397011,
        ('female', 'F', 'rejected'): 0.070068493150684918,
        ('male', 'A', 'admitted'): 0.11301369863013702,
        ('male', 'A', 'rejected'): 0.069266460450729109,
        ('male', 'B', 'admitted'): 0.077949624392399511,
        ('male', 'B', 'rejected'): 0.045779938135218703,
        ('male', 'C', 'admitted'): 0.026568714096332307,
        ('male', 'C', 'rejected'): 0.045238621299160404,
        ('male', 'D', 'admitted'): 0.030404330534688506,
        ('male', 'D', 'rejected'): 0.061730004418912916,
        ('male', 'E', 'admitted'): 0.011816173221387503,
        ('male', 'E', 'rejected'): 0.030384445426425107,
        ('male', 'F', 'admitted'): 0.0049447635881573011,
        ('male', 'F', 'rejected'): 0.077467962881131211
       }

方便起见，我们用一个 numpy 多元数组存储三元联合分布列。

In [8]:
# 建立变量取值和数组下标的映射
gender = dict(zip(['male', 'female'], range(2)))
department = dict(zip(['A', 'B', 'C', 'D', 'E', 'F'], range(6)))
admission = dict(zip(['admitted', 'rejected'], range(2)))

In [9]:
import itertools

# 初始化联合分布列
joint_probs = np.zeros((len(gender), len(department), len(admission)))

# 读入数据
for i, j, k in itertools.product(gender.keys(), department.keys(), admission.keys()):
    joint_probs[gender[i], department[j], admission[k]] = data[i, j, k]

In [10]:
joint_probs

array([[[0.113014, 0.069266],
        [0.07795 , 0.04578 ],
        [0.026569, 0.045239],
        [0.030404, 0.06173 ],
        [0.011816, 0.030384],
        [0.004945, 0.077468]],

       [[0.019567, 0.004295],
        [0.003756, 0.001768],
        [0.044547, 0.086474],
        [0.028999, 0.053856],
        [0.02084 , 0.065992],
        [0.005274, 0.070068]]])

如果我们要调取一位女性申请者被 'C' 院系录取的概率，用如下的语句即可

In [11]:
joint_probs[gender['female'], department['C'], admission['admitted']]

0.04454706142289001

下面我们看看会发生什么奇怪的事。首先我们算算变量的边缘分布。

In [12]:
probs_DA = joint_probs.sum(axis = 0)
probs_GA = joint_probs.sum(axis = 1)
probs_GD = joint_probs.sum(axis = 2)

这里 `probs_DA` 表示给定在 $D$ 和 $A$ 的值的情况下，将 $G$ 的所有值对应的概率相加，因此得到 $D$ 和 $A$ 的联合边缘分布，其中记录了被各院系总的录取率。此时如果我们想知道被 'C' 院系录取的概率是多少，可以输入

In [13]:
probs_DA[department['C'], admission['admitted']]

0.07111577551922231

再比如 `probs_GA` 是 $G$ 和 $A$ 的联合边缘分布，它是在给定 $G$ 和 $A$ 的值的情况下，将 $D$ 的所有值对应的概率相加得到的，表示某一性别总的录取率。例如，我们想知道女性的录取率，可以输入

In [14]:
probs_GA[gender['female'], admission['admitted']]

0.12298276623950503

下面我们算一算在一名申请者是女性的情况下，她被录取的概率是多少？
$$
p_{A|G}(\text{admitted }|\text{ female}) = \frac{p_{A,G}(\text{admitted}, \text{female})}{p_{G}(\text{female})}
$$

容易想到女性录取情况存储在 probs_GA 中。数据如下

In [15]:
female_only = probs_GA[gender['female']]

In [16]:
female_only

array([0.122983, 0.282452])

这表明女性被录取的概率为 0.12，被拒绝的概率为 0.28.

通过规一化，我们得到女性录取率的分布如下

In [17]:
female_admission_prob = female_only / np.sum(female_only)

In [18]:
female_admission_prob

array([0.303335, 0.696665])

为了方便使用，我们将上述分布列表示为映射形式。我们可以看到一位女性申请者有 30% 的概率被录取，70% 的概率被拒绝。

In [22]:
female_admission_prob = dict(zip(admission.keys(), female_admission_prob))

下面我们看看男性的情况

In [23]:
male_only = probs_GA[gender['male']]
male_admission_prob = male_only / np.sum(male_only)
male_admission_prob = dict(zip(admission.keys(), male_admission_prob))

print(male_admission_prob)

{'admitted': 0.44519509476031227, 'rejected': 0.5548049052396877}


我们看到一位男性申请者有 45% 的概率被录取，55% 的概率被拒绝。这是多面明显的歧视啊！

我们进一步考察各院系歧视情况的严重程度。

首先，我们看看在被录取的人中，男性和女性所占比例。

In [25]:
admitted_only = probs_GA[:, admission['admitted']]
admitted_gender_prob = admitted_only / np.sum(admitted_only)
admitted_gender_prob = dict(zip(gender.keys(), admitted_gender_prob))

print(admitted_gender_prob)

{'male': 0.6827725345369992, 'female': 0.3172274654630008}


可见，各院系更偏爱男性申请者。

下面，我们分院系考察上述问题

In [30]:
for d, g in itertools.product(department.keys(), gender.keys()):
    admission_prob = joint_probs[gender[g], department[d], :]
    admission_prob = admission_prob / np.sum(admission_prob)
    admission_prob = dict(zip(admission.keys(), admission_prob))
    print('一名{}性申请者被 {} 院系录取的概率为 {}.'.format({'male': '男', 'female': '女'}[g], d, admission_prob['admitted']))

一名男性申请者被 A 院系录取的概率为 0.6200000000000001.
一名女性申请者被 A 院系录取的概率为 0.8200000000000004.
一名男性申请者被 B 院系录取的概率为 0.6300000000000003.
一名女性申请者被 B 院系录取的概率为 0.679999999999997.
一名男性申请者被 C 院系录取的概率为 0.37000000000000005.
一名女性申请者被 C 院系录取的概率为 0.3400000000000001.
一名男性申请者被 D 院系录取的概率为 0.3300000000000004.
一名女性申请者被 D 院系录取的概率为 0.35.
一名男性申请者被 E 院系录取的概率为 0.27999999999999936.
一名女性申请者被 E 院系录取的概率为 0.23999999999999955.
一名男性申请者被 F 院系录取的概率为 0.05999999999999988.
一名女性申请者被 F 院系录取的概率为 0.0699999999999997.


神奇的事发生了，我们看到除了 C 和 E 两个院系，其余院系男性的录取率都低于女性，即便这两个院系，录取率相差也不大。

为什么总的录取率男性高于女性，但是各个院系的录取率女性高于男性呢？